Se usan perceptrones multicapa, propagación hacia adelante, backpropagation y actualización de pesos para el reconocimiento de números de [`reconocNumeros.ipynb`](https://colab.research.google.com/drive/1EdnmbkRiDWxfZXPL3deCUlc29j7sbcva#scrollTo=GN4Nk4GwfcTv). Se usa una RNA de capa oculta para la clasificación de números. Se utiliza ReLU en la capa oculta y softmax en la capa de salida para la predicción de probabilidades. El aprendizaje se realiza mediante bsckpropagation y optimización de los pesos con descenso de gradiente.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from funciones import getData, y2indicator, error_rate, relu, softmax

class NeuralNetwork:
    def __init__(self, input_size, hidden_size, output_size):
        self.W1 = np.random.randn(input_size, hidden_size) / np.sqrt(input_size)
        self.b1 = np.zeros(hidden_size)
        self.W2 = np.random.randn(hidden_size, output_size) / np.sqrt(hidden_size)
        self.b2 = np.zeros(output_size)
    def forward(self, X):
        Z = relu(X.dot(self.W1) + self.b1)
        A = softmax(Z.dot(self.W2) + self.b2)
        return Z, A
    def backward(self, X, Z, A, T, learning_rate=1e-4, reg=1e-3):
        N = X.shape[0]
        dA = A - T
        self.W2 -= learning_rate * (Z.T.dot(dA) + reg * self.W2)
        self.b2 -= learning_rate * (dA.sum(axis=0) + reg * self.b2)
        dZ = dA.dot(self.W2.T) * (Z > 0)
        self.W1 -= learning_rate * (X.T.dot(dZ) + reg * self.W1)
        self.b1 -= learning_rate * (dZ.sum(axis=0) + reg * self.b1)
    def train(self, X, Y, Xvalid, Yvalid, epochs=500, learning_rate=1e-4, reg=1e-3):
        T = y2indicator(Y)
        Tvalid = y2indicator(Yvalid)
        costs = []
        for i in range(epochs):
            Z, A = self.forward(X)
            self.backward(X, Z, A, T, learning_rate, reg)
            if i % 10 == 0:
                _, Avalid = self.forward(Xvalid)
                c = self.cost(Tvalid, Avalid)
                costs.append(c)
                e = error_rate(Yvalid, np.argmax(Avalid, axis=1))
                print(f"Epoch: {i}, Cost: {c}, Error: {e}")
        plt.plot(costs)
        plt.xlabel("Epochs")
        plt.ylabel("Cost")
        plt.show()
    def cost(self, T, Y):
        return -(T * np.log(Y)).sum()
    def predict(self, X):
        _, A = self.forward(X)
        return np.argmax(A, axis=1)
def main():
    Xtrain, Ytrain, Xvalid, Yvalid = getData()
    input_size = Xtrain.shape[1]
    hidden_size = 200
    output_size = len(set(Ytrain))
    model = NeuralNetwork(input_size, hidden_size, output_size)
    model.train(Xtrain, Ytrain, Xvalid, Yvalid, epochs=500)
    print("Final validation error rate:", error_rate(Yvalid, model.predict(Xvalid)))
    for i in range(10):
        img = Xvalid[i].reshape(28, 28)
        plt.imshow(img, cmap='gray')
        plt.show()
        print(f"Prediction: {model.predict(Xvalid[i].reshape(1, -1))}, Actual: {Yvalid[i]}")
if __name__ == '__main__':
    main()

ModuleNotFoundError: No module named 'funciones'